In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [2]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
    'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?'
    'type=data&subject=writing&grade=8&'
    'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'
)

example_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [4]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'serviceVersion': '10.24.2025.2',
 'dwellTimeMS': '4278.739',
 'avgWebHostCPUTotalLoad': 'N/A',
 'dataHitType': 'FROM_DATABASE',
 'Source': 'B11A',
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Sex',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Sex',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStat

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Sex,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Sex,2,Female,158.567105,1,0


### Wrap your long URL into a shorter version using the `params` argument

In [4]:
base_url = "https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx"

params = {
    "type": "data",
    "subject": "writing",
    "grade": "8",
    "subscale": "WRIRP",
    "variable": "GENDER",
    "jurisdiction": "NP",
    "stattype": "MN:MN",
    "Year": "2011"
}

response = requests.get(base_url, params=params)

print(response.url)

https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN%3AMN&Year=2011


In [5]:
pd.DataFrame(response.json()['result'])

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Sex,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Sex,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [6]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
    'https://www.nationsreportcard.gov/'
    'Dataservice/GetAdhocData.aspx?'
    'type=data&subject=writing&grade=8&'
    'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'
)

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [7]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [400]>

In [ ]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

JSONDecodeError: Invalid control character at: line 1 column 293 (char 292)

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [ ]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [ ]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [ ]:
# your code here

# 2. Example 2: needs credentials; no wrapper

Create an account here: https://site.financialmodelingprep.com/

In [ ]:
## get the key
API_KEY = "PASTE KEY HERE"

# Activity 2: Plot historical stock prices for a given company

- Look up the documentation: https://site.financialmodelingprep.com/developer/docs
- Find how you can get historical stock prices for a given company ticker symbol
- Write a function that takes in a ticker symbol and an API key and returns a DataFrame with the historical stock prices for that company
- Plot the closing prices over time for a NVIDIA, ticker symbol `NVDA`

In [ ]:
# your code here